In [1]:
import sys
from argparse import ArgumentParser
import numpy as np
import torch
import torch_geometric as tg

from load_dataset import get_LP_dataset, get_NC_dataset, get_GC_dataset
from set_up_training import set_up_LP_training, set_up_NC_training
from prepare_data import prepare_LP_data, prepare_NC_data
from utils import weights_init, seed_everything
from execution import NC_run, LP_run, GC_run
from model import AdamGNN, GAE, InnerProductDecoder

import warnings
warnings.filterwarnings('ignore')
# %matplotlib inline

In [2]:
sys.argv = ['']  # execution on jupyter notebook
parser = ArgumentParser()
# general
parser.add_argument('--task', dest='task', default='LP', type=str,
                    help='LP; NC; GC')
parser.add_argument('--data_name', dest='data_name', default='emails', type=str,
                    help='cora; emails; ...')
parser.add_argument('--mode', dest='mode', default='baseline', type=str,
                    help='experiment mode. E.g., baseline or basemodel')
parser.add_argument('--model', dest='model', default='GCN', type=str,
                    help='model class name. E.g., GCN, PGNN, HCGNN...')
parser.add_argument('--local_agg_gnn', dest='local_agg_gnn', default='GCN', type=str,
                    help='GNN model used for primary node generation')
parser.add_argument('--fitness_mode', dest='fitness_mode', default='both_c', type=str,
                    help='how do we jointly use two fitness scores. E.g., c: \phi_c s: \phi_s both_j both_c')
parser.add_argument('--pooling_mode', dest='pooling_mode', default='att', type=str,
                    help='adaptive pooling mode. E.g., mean, max, att')
parser.add_argument('--num_levels', dest='num_levels', default=4, type=int,
                    help='number of hierarchical levels.')
parser.add_argument('--hid_dim', dest='hid_dim', default=64, type=int,
                    help='the hidden dimentin of neural network')
parser.add_argument('--cluster_range', dest='cluster_range', default=1, type=int,
                    help='number of hops to define the ego-network')
parser.add_argument('--overlap', dest='overlap', default=True, type=bool,
                    help='whether allow overlapping between different selected ego-networks')
parser.add_argument('--loss_mode', dest='loss_mode', default='all', type=str,
                    help='mode of loss fucntion. E.g., task, KL, R')
parser.add_argument('--batch_size', dest='batch_size', default=64, type=int,
                    help='batch_size')
parser.add_argument('--use_attribute', dest='use_attribute', default=True, type=bool,
                    help='wheter adopt attributes of dataset')
parser.add_argument('--output_mode', dest='output_mode', default='ATT', type=str,
                    help='the mode of flyback aggregator. E.g., ATT, MEAN, MAC, LSTM')
parser.add_argument('--gat_head', dest='gat_head', default=0, type=int,
                    help='the number of attention head if use GAT for primary node embedding generation')
parser.add_argument('--all_cluster', dest='all_cluster', default=True, type=bool,
                    help='whether pick all ego-networks that satisfy selection requirements')
parser.add_argument('--pooling_ratio', dest='pooling_ratio', default=1, type=int,
                    help='the ratio of selection if do not select all satisfied ego-networks')
parser.add_argument('--l2_regularization', dest='l2_regularization', default=5e-4, type=float,
                    help='l2 regularization value')
parser.add_argument('--edge_threshold', dest='edge_threshold', default=0, type=float,
                    help='filter weak edges')
parser.add_argument('--do_view', dest='do_view', default=False, type=bool,
                    help='whether present detailed training process')
parser.add_argument('--early_stop', dest='early_stop', default=100, type=int,
                    help='patience to wait for training')

parser.add_argument('--gpu', dest='gpu', default=True, type=bool,
                    help='whether use gpu')
parser.add_argument('--seed', dest='seed', default=123, type=int)

# dataset
parser.add_argument('--num_train', dest='num_train', default=80, type=float)
parser.add_argument('--use_features', dest='use_features', default=True, type=bool,
                    help='whether use node features')

# model
parser.add_argument('--lr', dest='lr', default=1e-2, type=float)
parser.add_argument('--num_epoch', dest='num_epoch', default=201, type=int)
parser.add_argument('--verbose', dest='verbose', default=1, type=int)
parser.add_argument('--relu', dest='relu', default=True, type=bool)
parser.add_argument('--dropout', dest='dropout', default=False, type=bool)
parser.add_argument('--drop_ratio', dest='drop_ratio', default=0.5, type=float)

args = parser.parse_args()

In [3]:
args.data_name = 'emails' # emails cora citeseer pubmed cs phisic computers photo 
args.lr = 0.01
args.num_train = 80
args.local_agg_gnn = 'GCN' # GCN SAGE GIN
args.gat_head = 0
args.fitness_mode = 'both_c' # c: connect s: structure both_j both_c
args.pooling_mode = 'att' # mean att max
args.num_levels = 3 # 1, 2, 3
args.encoder_layers = 1
args.hid_dim = 64
args.overlap = True
args.cluster_range = 1
args.drop_ratio = 0.9
args.loss_mode = 'kl' # lp kl
args.output_mode = 'ATT' # ATT GCN GAT MEAN MAX LINEAR LSTM NONE
args.seed = 4329
args.edge_threshold = 0
args.all_cluster = True
args.pooling_ratio = 0.1
args.batch_size = None
args.dropout = True # no necessary
args.relu = True # optional for shallow layers
args.do_view = True

args.device = torch.device('cuda:' + str(0) if args.gpu and torch.cuda.is_available() else 'cpu')
seed_everything(args.seed)
print(args)

Namespace(all_cluster=True, batch_size=None, cluster_range=1, data_name='emails', device=device(type='cuda', index=0), do_view=True, drop_ratio=0.9, dropout=True, early_stop=100, edge_threshold=0, encoder_layers=1, fitness_mode='both_c', gat_head=0, gpu=True, hid_dim=64, l2_regularization=0.0005, local_agg_gnn='GCN', loss_mode='kl', lr=0.01, mode='baseline', model='GCN', num_epoch=201, num_levels=3, num_train=80, output_mode='ATT', overlap=True, pooling_mode='att', pooling_ratio=0.1, relu=True, seed=4329, task='LP', use_attribute=True, use_features=True, verbose=1)


In [4]:
# load dataset
data = get_LP_dataset(
    dataset_name=args.data_name,
    use_features=args.use_attribute,
    seed=args.seed
)

# set up training mode
graph, df_train, df_valid, df_test = set_up_LP_training(
    data=data, num_train=args.num_train, seed=args.seed
)
# prepare data
data = prepare_LP_data(
    data=data,
    df_train=df_train, df_valid=df_valid, df_test=df_test,
    batch_size=args.batch_size, device=args.device
)

is reading emails dataset...
Graph has 799 nodes and 9662 edges.
we need to collect 1932 negative edges.
Generating negative instances uses 0.08 seconds.
We need to remove 1932 edges.
Generating positive instances uses 0.06 seconds.
train pos 7730, train neg 7730 valid pos 966, valid neg 966, test pos 966, test neg 966


## experiments

In [5]:
encoder = AdamGNN(
    args=args, feat_dim=data.x.shape[1], out_dim=args.hid_dim
)
model = GAE(encoder=encoder, decoder=InnerProductDecoder())
model = model.to(args.device)
print(model)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=args.lr, weight_decay=args.l2_regularization
)

GAE(
  (encoder): AdamGNN(
    (encoder): Encoder(
      (convs): ModuleList(
        (0): ModuleList(
          (0): GCNConv(799, 64)
        )
        (1): ModuleList(
          (0): GCNConv(64, 64)
        )
        (2): ModuleList(
          (0): GCNConv(64, 64)
        )
      )
    )
    (pools): ModuleList(
      (0): Adaptive_pooling(
        (score_lin): Linear(in_features=128, out_features=1, bias=True)
        (pool_lin): Linear(in_features=128, out_features=1, bias=True)
      )
      (1): Adaptive_pooling(
        (score_lin): Linear(in_features=128, out_features=1, bias=True)
        (pool_lin): Linear(in_features=128, out_features=1, bias=True)
      )
    )
    (out_cat): Merge_xs(
      (lin_att): Linear(in_features=128, out_features=1, bias=True)
    )
    (last_layer): Linear(in_features=64, out_features=64, bias=True)
  )
  (decoder): InnerProductDecoder()
)


In [6]:
LP_run(
    args=args, model=model, optimizer=optimizer, data=data
)


Epoch 0 starts !
--------------------------------------------------------------------------------
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 439 nodes, 36 ego nodes, 403 remaining nodes
Level 1 has 9229 edges.
Origin graph has 439 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:439, transform from 439 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transform from 414 to 439
after, embedding shape: 439
learned embedding shape:439, transform from 439 to 799
after, embedding shape: 799
1.4070758819580078 0.003196174278855324
1.4102720022201538
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform fr

Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transform from 414 to 438
after, embedding shape: 438
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
1.1519514322280884 0.0037675220519304276
1.1557189226150513
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transform from 414 to 438
after, embedding shape: 438
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Evaluating Epoch 6, tim

Evaluating Epoch 12, time 0.212
train ROC-AUC = 0.9188, valid ROC-AUC = 0.9028, Test ROC-AUC = 0.9149
best valid AUC performance is 0.9156, best test performance is 0.9152 and epoch_id is 3

Epoch 13 starts !
--------------------------------------------------------------------------------
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transform from 414 to 438
after, embedding shape: 438
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
1.0348777770996094 0.004146271850913763
1.039023995399475
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 40

Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transform from 414 to 438
after, embedding shape: 438
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Evaluating Epoch 19, time 0.204
train ROC-AUC = 0.9475, valid ROC-AUC = 0.9403, Test ROC-AUC = 0.9436
best valid AUC performance is 0.9403, best test performance is 0.9436 and epoch_id is 19

Epoch 20 starts !
--------------------------------------------------------------------------------
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated 

0.9506211876869202 0.0035388267133384943
0.9541600346565247
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transform from 414 to 438
after, embedding shape: 438
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Evaluating Epoch 25, time 0.227
train ROC-AUC = 0.9485, valid ROC-AUC = 0.9400, Test ROC-AUC = 0.9401
best valid AUC performance is 0.9452, best test performance is 0.9471 and epoch_id is 20

Epoch 26 starts !
--------------------------------------------------------------------------------
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes,

...

0.8793174028396606
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transform from 414 to 438
after, embedding shape: 438
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Evaluating Epoch 169, time 0.211
train ROC-AUC = 0.9775, valid ROC-AUC = 0.9489, Test ROC-AUC = 0.9554
best valid AUC performance is 0.9513, best test performance is 0.9584 and epoch_id is 147

Epoch 170 starts !
--------------------------------------------------------------------------------
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 

0.8875107765197754 0.004112933296710253
0.8916237354278564
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transform from 414 to 438
after, embedding shape: 438
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Evaluating Epoch 175, time 0.218
train ROC-AUC = 0.9784, valid ROC-AUC = 0.9485, Test ROC-AUC = 0.9585
best valid AUC performance is 0.9513, best test performance is 0.9584 and epoch_id is 147

Epoch 176 starts !
--------------------------------------------------------------------------------
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego node

Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transform from 414 to 438
after, embedding shape: 438
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
0.882103443145752 0.00372455851174891
0.8858280181884766
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transf

Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transform from 414 to 438
after, embedding shape: 438
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Evaluating Epoch 188, time 0.218
train ROC-AUC = 0.9771, valid ROC-AUC = 0.9459, Test ROC-AUC = 0.9530
best valid AUC performance is 0.9550, best test performance is 0.9604 and epoch_id is 182

Epoch 189 starts !
--------------------------------------------------------------------------------
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolat

Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, transform from 414 to 438
after, embedding shape: 438
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
0.875524640083313 0.004063136409968138
0.8795877695083618
Level 0 has 15460 edges.
Origin graph has 799 nodes, 32 isolated nodes
Pooled graph has 438 nodes, 36 ego nodes, 402 remaining nodes
Level 1 has 9226 edges.
Origin graph has 438 nodes, 32 isolated nodes
Pooled graph has 414 nodes, 2 ego nodes, 412 remaining nodes
learned embedding shape:438, transform from 438 to 799
after, embedding shape: 799
Level 2 has 7790 edges.
learned embedding shape:414, trans